# Projeto 1 - Ciência dos Dados

Nome: Fabrizio Stresser Milan

Nome: Tomás Fiorelli Barbosa

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [38]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from nltk.tokenize import TweetTokenizer

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [39]:
filename = 'Cyberpunk 2077.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Cyberpunk 2077.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [40]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificador de tweets
0,@alexia_rks alexia caralho\namei demais mano t...,0
1,muitos vídeos massas da @superkikachu pra mim ...,1
2,cyberpunk so vai sair em 2077 mesmo ne,0
3,cyberpunk 2077 vai causar um impacto monumenta...,1
4,bem agora e so segurar as pontas ate novembro ...,1


In [41]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação Manual
0,#voxelart rt @voxeloficial: desenvolvedores de...,1
1,"@sonicuzin não, cyberpunk vai lançar em 2077 ó...",0
2,rt @cpfuturosombrio: quando falamos que cyberp...,1
3,rt @alexzitoooo: graças a deus vou jogar o cyb...,1
4,acho que só vou esperar mandalorian e cyberpun...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o jogo Cyberpunk 2077, videogame em produção desde 2012 que será lançado no dia 19/11/2020 para as plataformas XBOX, PlayStation e PC. Consideramos como tweets relevantes aqueles que falaram bem a respeito do jogo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [42]:

# ----- Função para a limpeza dos textos -----
import re 

def cleanup(text):
    punctuation = '''[@!?:\n;"'(){}/.,]'''
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
# --------------------------------------------

# ----- Transformador em Texto Corrido -----
relevante = train.loc[train['Classificador de tweets'] == 1, ['Treinamento']]
irrelevante = train.loc[train['Classificador de tweets'] == 0, ['Treinamento']]

txt_rel = ''
for i in np.arange(len(relevante['Treinamento'])):
    txt_rel += relevante.iloc[i, 0]

txt_irr = ''
for i in np.arange(len(irrelevante['Treinamento'])):
    txt_irr += irrelevante.iloc[i, 0]

todas_palavras = txt_rel + txt_irr
# -------------------------------------------

# Listas de Palavras
list_rel = cleanup(txt_rel).split()
list_irr = cleanup(txt_irr).split()
list_palavras = cleanup(todas_palavras).split()

# Séries de Palavras
serie_rel = pd.Series(list_rel)
serie_irr = pd.Series(list_irr)
serie_palavras = pd.Series(list_palavras)

# Tabelas de Probabilidades Absolutas
tabela_absoluta_rel = serie_rel.value_counts()
tabela_absoluta_irr = serie_irr.value_counts()
tabela_absoluta_palavras = serie_palavras.value_counts()

# Tabelas de Probabilidades Relativas
tabela_relativa_rel = serie_rel.value_counts(True)
tabela_relativa_irr = serie_irr.value_counts(True)
tabela_relativa_palavras = serie_palavras.value_counts(True)

In [43]:
# ----- Função Classificadora de Tweets -----
def classificador(texto):
    texto_minusculo = texto.lower()
    texto_limpo = (cleanup(texto_minusculo))
    tweet_tokenizer = TweetTokenizer()
    lista_palavras2 = tweet_tokenizer.tokenize(texto_limpo)

    P_F_Rel = 1
    P_F_Irr = 1
    
    # Verifica se todas as palavras de um tweet existem em suas respectivas listas de palavras
    for i in lista_palavras2:
        if i in list_rel:
            verifica_lista_rel = True
        else:
            verifica_lista_rel = False
            break
    
    for i in lista_palavras2:
        if i in list_irr:
            verifica_lista_irr = True
        else:
            verifica_lista_irr = False
            break
    
    # Cálculo das Probabilidades - Relevantes
    if verifica_lista_rel:
        for i in lista_palavras2:
            P_F_Rel = P_F_Rel*(tabela_relativa_rel[str(i)])
    else:
        # laplace Smoothing
        for i in lista_palavras2:
            if i in list_rel:
                P_F_Rel = P_F_Rel*((tabela_absoluta_rel[str(i)]+1)/(len(tabela_absoluta_rel)+len(tabela_absoluta_palavras)))
            else:
                P_F_Rel = P_F_Rel*(1/(len(tabela_absoluta_rel)+len(tabela_absoluta_palavras)))

    # Cálculo das Probabilidades - Irrelevante
    if verifica_lista_irr:
        for i in lista_palavras2:
            P_F_Irr = P_F_Irr*(tabela_relativa_irr[str(i)])
    else:
        # Laplace Smoothing
        for i in lista_palavras2:
            if i in list_irr:
                P_F_Irr = P_F_Irr*((tabela_absoluta_irr[str(i)]+1)/(len(tabela_absoluta_irr)+len(tabela_absoluta_palavras)))
            else:
                P_F_Irr = P_F_Irr*(1/(len(tabela_absoluta_irr)+len(tabela_absoluta_palavras)))
    
    # Resultado final
    if P_F_Rel > P_F_Irr:
        return(1)
    else:
        return(0)
# -------------------------------------------

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [44]:
# ----- Atribuindo Classificação para os Tweets -----
resultado = []
for i in range(len(test)):
    resultado.append(classificador(test.iloc[i, 0]))

novo_test = test
novo_test['Classificação por ML'] = resultado
# ---------------------------------------------------

In [45]:
# ----- Contabilizando e Comparando as Classificações -----
verdadeiro_positivo = 0
falso_positivo = 0
verdadeiro_negativo = 0
falso_negativo = 0

for i in range(len(novo_test)):
    if novo_test.iloc[i, 1] == 1:
        if novo_test.iloc[i, 2] == 1:
            verdadeiro_positivo += 1
        else:
            falso_negativo += 1
    else:
        if novo_test.iloc[i, 2] == 0:
            falso_positivo += 1
        else:
            verdadeiro_negativo += 1
# --------------------------------------------------------

In [46]:
# ----- Resultado Final -----
print('''
Tweets classificados como:

- Verdadeiros positivos: {}%
- Falsos positivos: {}%
- Verdadeiros negativos: {}%
- Falsos negativos: {}%
'''.format(verdadeiro_positivo/2, falso_positivo/2, verdadeiro_negativo/2, falso_negativo/2))

# O valor foi dividido por 2, porque estamos dividindo por 200 (número total de tweets) e 
# multiplicando por 100 (para colocar em porcentagem).

# ---------------------------


Tweets classificados como:

- Verdadeiros positivos: 61.5%
- Falsos positivos: 1.5%
- Verdadeiros negativos: 37.0%
- Falsos negativos: 0.0%



___
### Concluindo

Após realizar o trabalho, nós chegamos a conclusão de que o classificador de tweets foi um sucesso, já que entre todos os tweets analisados cerca de 98,5% foram corretamente analisados pelo classificador, enquanto apenas 1,5% dos tweets obtiveram uma análise incorreta. Esta ocorreu pois o classificador possui as suas falhas devido a simplicidade do mesmo, como por exemplo, a fraca interpretação de mensagens sarcásticas, já que em um dos tweets é dito que "Cyberpunk só vai sair em 2077 mesmo, né?!" e essa mensagem faz uma crítica aos inúmeros adiamentos para o lançamento do jogo, entretanto o classificador não consegue ter esta interpretação e atribuiu a esse tweet o valor "relevante", mesmo ele sendo classificado manualmente como irrelevante.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**